# Estimando distribuciones (parte 1)
El objetivo de esta notebook es explorar una primera manera de aproximar $p(y|x)$ y $p(x|y)$ en un set de datos tabular. En este set de datos $x$ tiene valores discretos, $x\in\mathbb{D}^k$, y el target $y$ es un booleano, $y\in\{0,1\}$.

## Imports

In [1]:
import numpy as np
import pandas as pd
import random

## Cargamos los datos

In [2]:
df = pd.read_csv('./tennis.csv', delimiter=',', header=0)
df

,Day,Outlook,Temp,Humidity,Wind,Tennis
0,D1,Sunny,Hot,High,Weak,No
1,D2,Sunny,Hot,High,Strong,No
2,D3,Overcast,Hot,High,Weak,Yes
3,D4,Rain,Mild,High,Weak,Yes
4,D5,Rain,Cool,Normal,Weak,Yes
5,D6,Rain,Cool,Normal,Strong,No
6,D7,Overcast,Cool,Normal,Strong,Yes
7,D8,Sunny,Mild,High,Weak,No
8,D9,Sunny,Cool,Normal,Weak,Yes
9,D10,Rain,Mild,Normal,Weak,Yes


### Eliminamos la columna Day 

In [3]:
df = df.drop('Day', axis=1)
df

,Outlook,Temp,Humidity,Wind,Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [4]:
X_names = df.columns.to_list()[:-1]
X_names

['Outlook', 'Temp', 'Humidity', 'Wind']

Guardamos en la variable $X$ todas las features del dataset.

In [5]:
X = df.iloc[:,0:-1]
X

,Outlook,Temp,Humidity,Wind
0,Sunny,Hot,High,Weak
1,Sunny,Hot,High,Strong
2,Overcast,Hot,High,Weak
3,Rain,Mild,High,Weak
4,Rain,Cool,Normal,Weak
5,Rain,Cool,Normal,Strong
6,Overcast,Cool,Normal,Strong
7,Sunny,Mild,High,Weak
8,Sunny,Cool,Normal,Weak
9,Rain,Mild,Normal,Weak


In [6]:
Y_name = df.columns.to_list()[-1]
Y_name

'Tennis'

Guardamos en $Y$ el objetivo

In [7]:
Y = df.iloc[:,-1]
Y

0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Tennis, dtype: object

## Construimos una tabla de observaciones

En este paso vamos a crear una tabla de observaciones. Esta tabla tiene que contener la frecuencia de cada observación. Para este ejemplo tomaremos a $x$ como **Outlook**.

Calcule las dimensiones de la tabla

In [8]:
# Cantidad total de elementos
N = df.shape[0]

# Elementos únicos de la clase Outlook
xvalues = df.iloc[:,0].unique().tolist()
dimx = len(xvalues)

# Elementos únicos del objetivo
yvalues = df.iloc[:,-1].unique().tolist()
dimy = len(yvalues)

print(f'Cantidad total de elementos: {N}')
print(f'Elementos únicos de la clase Outlook: {xvalues}')
print(f'Elementos únicos del objetivo: {yvalues}')

Cantidad total de elementos: 14
Elementos únicos de la clase Outlook: ['Sunny', 'Overcast', 'Rain']
Elementos únicos del objetivo: ['No', 'Yes']


Calculamos la tabla de frecuencia.

In [9]:
obs = pd.DataFrame(0, columns=yvalues, index=xvalues)

In [10]:
obs

,No,Yes
Sunny,0,0
Overcast,0,0
Rain,0,0


In [16]:
obs = pd.DataFrame(0, columns=yvalues, index=xvalues)

## Llene la tabla de observaciones
for i in range(dimx):
    for j in range(dimy):
        obs.iloc[i,j] = df[(df.iloc[:,0] == xvalues[i]) & (df.iloc[:,-1] == yvalues[j])].shape[0]

obs

,No,Yes
Sunny,3,2
Overcast,0,4
Rain,2,3


## Aproximación de la distribución conjunta $p(x,y)$

Tome a $x$ como Outlook y aproxime la distribución conjunta utilizando la tabla de observaciones. 

In [17]:
joint_x_y = obs / N
joint_x_y

,No,Yes
Sunny,0.214286,0.142857
Overcast,0.000000,0.285714
Rain,0.142857,0.214286


1. ¿Qué significa el valor calculado en los índices "Sunny", "Yes"?

Hay un 14,3% de probabilidad de que se juegue al tennis en un día soleado

2. ¿Justifique el resultado de la pareja "Overcast", "No"?

Dado que el enfoque que tomamos es frecuentista, al no tener registros en los que Outlook sea Overcast y no se haya jugado al tennis, la probabilidad conjunta es 0

## Aproximamos $p(y|x)$

Tome a $x$ como **Outlook** y estime la probabilidad condicional de $y$ dado $x$. Luego realice una muestra de 10 valores de $y$ dado $x = Sunny$.

Calculamos la cantidad de entradas por cada valor distinto de $x$.

In [18]:
m = obs.sum(axis=1)
obs['m'] = m
m

Sunny       5
Overcast    4
Rain        5
dtype: int64

Calculamos la cantidad de entradas por cada valor distinto de $y$.

In [19]:
l = obs.sum(axis=0)
obs.loc['l'] = l
l

No      5
Yes     9
m      14
dtype: int64

In [20]:
obs

,No,Yes,m
Sunny,3,2,5
Overcast,0,4,4
Rain,2,3,5
l,5,9,14


Calcule la probabilidad condicional de $y$ dado $x$.

In [ ]:
p_y_x = pd.DataFrame(0.0, columns=yvalues, index=xvalues)

## Llene la tabla de probabilidades condicionales p(y|x)

for x in xvalues:
    for y in yvalues:
        p_y_x.loc[x, y] = obs.loc[x, y] / obs.loc[x, 'm']

p_y_x

,No,Yes
Sunny,0.6,0.4
Overcast,0.0,1.0
Rain,0.4,0.6


3. ¿La suma de cada fila siempre tiene que dar 1? ¿Por qué?

    Si, porque dado que al fijar la X en un valor, por ejemplo, X = "Sunny", los 2 escenarios posibles son que Y = "Yes" o Y = "No", y estas son los 2 valores que aparecen en la primera fila, por lo que tienen que sumar 1

4. ¿Y si la suma de las columnas?

    No, la suma de las columnas no tiene que sumar 1, y no hace ningun sentido sumarlas

Realice 10 muestras de $y$ dado $x = Sunny$.

Puede utilizar la función random.choice de numpy. https://numpy.org/doc/stable/reference/random/generated/numpy.random.choice.html

In [25]:
sampled_values = np.random.choice(yvalues,size=10,p=p_y_x.loc['Sunny'].values)

sampled_values

array(['Yes', 'No', 'No', 'No', 'No', 'Yes', 'Yes', 'No', 'No', 'No'],
      dtype='<U3')

5. ¿Qué pasaría si utilizamos $x = Overcast$ en vez de $x = Sunny$? ¿Tiene sentido que pase esto? ¿Por qué?

    Si $x = Overcast$ las 10 muestras sampleadas serán Yes, ya que la probabilidad de que se juegue Tennis, sabiendo que $Outlook = Overcast% es 100%

## Aproximamos $p(x|y)$
Tome a $x$ como Outlook y estime la probabilidad condicional de $x$ dado $y$ basandose en la tabla de observaciones. Luego realice 10 muestras de $x$ dado $y = Yes$.

$p(x|y)$

In [27]:
p_x_y = pd.DataFrame(0.0, columns=yvalues, index=xvalues)

## Llene la tabla de probabilidades condicionales p(x|y)
for x in xvalues:
    for y in yvalues:
        p_x_y.loc[x, y] = obs.loc[x, y] / obs.loc['l', y]

p_x_y

,No,Yes
Sunny,0.6,0.222222
Overcast,0.0,0.444444
Rain,0.4,0.333333


In [31]:
p_x_y

# Obtenemos las probabilidades de x cuando y es Yes
p_x_y.loc[:, 'Yes']

Sunny       0.222222
Overcast    0.444444
Rain        0.333333
Name: Yes, dtype: float64

Muestreo

In [34]:
sampled_values = np.random.choice(xvalues,size=10,p=p_x_y.loc[:, 'Yes'].values)

sampled_values

array(['Sunny', 'Overcast', 'Rain', 'Rain', 'Overcast', 'Rain', 'Sunny',
       'Rain', 'Overcast', 'Rain'], dtype='<U8')

## Aproxime $p(y,o,h,w,t)$

Aproxime la proabilidad conjunta del Tennis (y), Outlook (o), Humidity (h), Wind (w), Temp (t).

Recuerde que $p(y,o,h,w,t)$ = $p(y)$.$p(o|y)$.$p(h|y,o)$.$p(w|y,o,h)$.$p(t|y,o,h,w)$

Calcule P(y)

In [37]:
# P(Y)

p_y = pd.DataFrame(0.0, columns=['Yes', 'No'], index=['l'])

## Llene la tabla de probabilidades marginales p(y)
for y in ['Yes', 'No']:
    p_y.loc[:, y] = obs.loc[:, y] / obs.loc[:, 'm']

p_y

,Yes,No
l,0.642857,0.357143


Calcule P(o|y)

In [39]:
p_o_y = p_x_y.copy()
p_o_y

,No,Yes
Sunny,0.6,0.222222
Overcast,0.0,0.444444
Rain,0.4,0.333333


Calcule P(h|y,o)

Recomendamos usar la función *crosstab* de pandas. En este link pueden encontrar un ejemplo de su uso: https://www.geeksforgeeks.org/pandas-crosstab-function-in-python/

In [47]:
df

,Outlook,Temp,Humidity,Wind,Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


Aproxime la proabilidad conjunta del Tennis (y), Outlook (o), Humidity (h), Wind (w), Temp (t).

In [48]:
y = df.loc[:,'Tennis']
o = df.loc[:,'Outlook']
h = df.loc[:,'Humidity']
w = df.loc[:,'Wind']
t = df.loc[:,'Temp']

In [51]:
# Calcule la tabla de frecuencia
p_h_yo = pd.crosstab([y,o],h,rownames=['Tennis','Outlook'], colnames=['Humidity'])

# Luego se divide cada fila por la suma de sus elementos, puede usar las funciones div y sum de pandas
p_h_yo = p_h_yo.div(p_h_yo.sum(axis=1), axis=0)

# No se oliden de llenar los valores NaN!!!
p_h_yo = p_h_yo.fillna(0)

# Cambiamos los nombres de los indices (si es que lo precisa)
p_h_yo.index = p_h_yo.index.set_names(['Tennis','Outlook'])
p_h_yo.columns = p_h_yo.columns.set_names(['Humidity'])

p_h_yo

Humidity             High    Normal
Tennis Outlook                     
No     Rain      0.500000  0.500000
       Sunny     1.000000  0.000000
Yes    Overcast  0.500000  0.500000
       Rain      0.333333  0.666667
       Sunny     0.000000  1.000000

Calcule P(w|y,o,h)

In [54]:
p_w_yoh = pd.crosstab([y,o,h],w,rownames=['Tennis','Outlook','Humidity'], colnames=['Wind'])
p_w_yoh = p_w_yoh.div(p_w_yoh.sum(axis=1), axis=0)
p_w_yoh

Wind                        Strong      Weak
Tennis Outlook  Humidity                    
No     Rain     High      1.000000  0.000000
                Normal    1.000000  0.000000
       Sunny    High      0.333333  0.666667
Yes    Overcast High      0.500000  0.500000
                Normal    0.500000  0.500000
       Rain     High      0.000000  1.000000
                Normal    0.000000  1.000000
       Sunny    Normal    0.500000  0.500000

Calcule P(t|y,o,h,w)

In [141]:
p_t_yohw = pd.crosstab([y,o,h,w],t,rownames=['Tennis','Outlook','Humidity','Wind'], colnames=['Temp'])
p_t_yohw = p_t_yohw.div(p_t_yohw.sum(axis=1), axis=0)
p_t_yohw

Temp                             Cool  Hot  Mild
Tennis Outlook  Humidity Wind                   
No     Rain     High     Strong   0.0  0.0   1.0
                Normal   Strong   1.0  0.0   0.0
       Sunny    High     Strong   0.0  1.0   0.0
                         Weak     0.0  0.5   0.5
Yes    Overcast High     Strong   0.0  0.0   1.0
                         Weak     0.0  1.0   0.0
                Normal   Strong   1.0  0.0   0.0
                         Weak     0.0  1.0   0.0
       Rain     High     Weak     0.0  0.0   1.0
                Normal   Weak     0.5  0.0   0.5
       Sunny    Normal   Strong   0.0  0.0   1.0
                         Weak     1.0  0.0   0.0

Calcule P(y,o,h,w,t)

Aproxime la proabilidad conjunta del Tennis (y), Outlook (o), Humidity (h), Wind (w), Temp (t).

Recuerde que $p(y,o,h,w,t)$ = $p(y)$.$p(o|y)$.$p(h|y,o)$.$p(w|y,o,h)$.$p(t|y,o,h,w)$

In [64]:
p_o_y

,No,Yes
Sunny,0.6,0.222222
Overcast,0.0,0.444444
Rain,0.4,0.333333


In [87]:
# Definimos una función que nos calcula la probabilidad conjunta usando la regla del producto.
def calculate_prob(y,o,h,w,t):
    "Calculate the probability of occurrence of a row"
    prob_y = p_y.loc[:,y].values[0]
    p_o_dado_y = p_o_y.loc[o,y]
    p_h_dado_y_o = p_h_yo.loc[(y,o),[h]].values[0]
    p_w_dado_y_o_h = p_w_yoh.loc[(y,o,h),[w]].values[0]
    p_t_dado_y_o_h_w = p_t_yohw.loc[(y,o,h,w),[t]].values[0]
    return prob_y * p_o_dado_y * p_h_dado_y_o * p_w_dado_y_o_h * p_t_dado_y_o_h_w

In [88]:
prob = calculate_prob('Yes','Sunny','Normal','Weak','Cool')

print(f'P(Yes|Sunny,Normal,Weak,Cool) = {prob}')
print(f"Cantidad de observaciones: {prob*N}")

P(Yes|Sunny,Normal,Weak,Cool) = 0.07142857142857142
Cantidad de observaciones: 1.0


Definimos el muestreo de datos completos.

Primero generamos un *y* utilizando *p(y)* y luego seguimos con las probabilidades condicionales.

In [145]:
def sample_from_y(y):
    "Generates a sample of weather conditions based on a specific y"
    o = np.random.choice(p_o_y.index.to_numpy(), p=p_o_y.loc[:,y].to_numpy(dtype=float))
    h = np.random.choice(p_h_yo.columns.to_numpy(), p=p_h_yo.loc[(y,o),:].to_numpy(dtype=float))
    w = np.random.choice(p_w_yoh.columns.to_numpy(), p=p_w_yoh.loc[(y,o,h),:].to_numpy(dtype=float))
    t = np.random.choice(p_t_yohw.columns.to_numpy(), p=p_t_yohw.loc[(y,o,h,w),:].to_numpy(dtype=float))
    return (y, o, h, w, t)

def sample():
    "Generates a sample of weather conditions based on a random y"
    y = np.random.choice(p_y.columns.to_numpy(), p=p_y.iloc[0].to_numpy(dtype=float))
    return sample_from_y(y)

In [146]:
samples = np.array([sample() for _ in range(len(df))])
new_df = pd.DataFrame(samples, columns=['Tennis', 'Outlook', 'Humidity', 'Wind', 'Temp'])
new_df

,Tennis,Outlook,Humidity,Wind,Temp
0,Yes,Sunny,Normal,Weak,Cool
1,Yes,Overcast,High,Strong,Mild
2,Yes,Rain,High,Weak,Mild
3,Yes,Sunny,Normal,Strong,Mild
4,Yes,Rain,High,Weak,Mild
5,No,Sunny,High,Weak,Mild
6,No,Sunny,High,Weak,Hot
7,No,Rain,High,Strong,Mild
8,Yes,Overcast,High,Strong,Mild
9,Yes,Overcast,Normal,Weak,Hot
